In [ ]:
!unzip cars_images.zip -d ./cars_images

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_JQJ.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_JxB.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_KQU.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_LBa.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_lCR.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_lVz.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_NDU.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_nUu.jpg  
  inflating: ./cars_images/Porsche_718_2019_82_20_360_25_4_70_50_173_19_RWD_2_2_Convertible_Oak.jpg  
  inflating: ./cars_images/Porsche_718_2019_82

In [ ]:
import argparse
import math
from pathlib import Path
import random
import sys
import json
from sklearn.preprocessing import OrdinalEncoder

import numpy as np
import tensorflow as tf

# Default data paths.
DEFAULT_TRAIN_INPUT_DIR = Path('./cars_images')
DEFAULT_TEST_INPUT_DIR = Path('./test_images')
DEFAULT_OUTPUT_DIR = Path('./tfrecords-output')
DEFAULT_NUM_SHARDS_TRAIN = 1
DEFAULT_NUM_SHARDS_TEST = 0

args = None

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--train-input-dir', type=Path, dest='train_input_dir',
                    default=DEFAULT_TRAIN_INPUT_DIR,
                    help='Train input directory to convert image files.')
parser.add_argument('--test-input-dir', type=Path, dest='test_input_dir',
                    default=DEFAULT_TEST_INPUT_DIR,
                    help='Test input directory to convert image files.')
parser.add_argument('--output-dir', type=Path, dest='output_dir',
                    default=DEFAULT_OUTPUT_DIR,
                    help='Output directory to store TFRecords files.')
parser.add_argument('--num-shards-train', type=int, dest='num_shards_train',
                    default=DEFAULT_NUM_SHARDS_TRAIN,
                    help='Number of shards to divide training set '
                         'TFRecords into.')
parser.add_argument('--num-shards-test', type=int, dest='num_shards_test',
                    default=DEFAULT_NUM_SHARDS_TEST,
                    help='Number of shards to divide test set '
                         'TFRecords into.')
args = parser.parse_args('')

In [ ]:
class TFRecordsConverter(object):
    """Class that handles converting images to TFRecords."""

    def __init__(self):

        self.train_input_path = args.train_input_dir
        self.test_input_path = args.test_input_dir
        self.output_path = args.output_dir
        self.num_shards_train = args.num_shards_train
        self.num_shards_test = args.num_shards_test
        self.output_path.mkdir(exist_ok = True)
        self.num_files_train = 0
        self.encoder = OrdinalEncoder()

        # Get lists of images and labels.
        self.train_filenames, self.train_labels = self.process_image_and_label(self.train_input_path)

        # Counter for total number of images processed.
        self.counter = 0

    def process_image_and_label(self, input_path):

        # Build the lists.
        images = []
        im_files = list(input_path.glob('*.jpg'))
        for im_path in im_files:
            images.append(im_path)

        self.num_files_train = len(images)

        # Randomize the order of all the images/labels.
        shuffled_indices = list(range(self.num_files_train))
        random.seed(12121)
        random.shuffle(shuffled_indices)

        filenames = [images[i] for i in shuffled_indices]
        carnames = np.array([file.name.split('_')[0] for file in filenames])
        carnames = carnames.reshape(-1, 1)

        self.encoder.fit(carnames)
        category_list = list(self.encoder.categories_[0])
        labels = self.encoder.transform(carnames).squeeze().astype(int)

        json_file = self.output_path / 'dataset.json'
        json_dict = {'num_train_images': self.num_files_train, 'category_list': category_list}

        with json_file.open('w') as f:
            json.dump(json_dict, f)

        return filenames, labels

    def write_tfrecords_file(self, output_path, filenames, labels, indices):
        """Writes out TFRecords file."""
        writer = tf.io.TFRecordWriter(str(output_path))
        for i in indices:
            filename = filenames[i]
            try:
                im_data = filename.read_bytes()
            except:
                print("The image file cannot be read. ")
                sys.exit()
            label = labels[i]

            # Example is a data format that contains a key-value store, where
            # each key maps to a Feature message. In this case, each Example
            # contains two features. One will be a ByteList for the raw image
            # data and the other will be an Int64List containing the index of
            # the corresponding label in the labels list from the file.
            example = tf.train.Example(features=tf.train.Features(feature={
                'filename': _bytes_feature(bytes(filename.name, encoding = "utf-8")),
                'image/encoded': _bytes_feature(im_data),
                'label': _int64_feature(label)}))
            writer.write(example.SerializeToString())
            self.counter += 1
            if not self.counter % 1000:
                print('Processed {} images...'.format(self.counter))
        writer.close()

    def convert(self):
        """This function will drive the conversion to TFRecords.

        Here, we partition the data into a training and testing set, then
        divide each data set into the specified number of TFRecords shards.
        """

        print('Processing training set TFRecords...')
        files_per_shard = int(math.ceil(self.num_files_train /
                                        self.num_shards_train))
        start = 0
        for i in range(0, self.num_shards_train):
            shard_path = self.output_path / f'train-{i}.tfrecords'
            # Get a subset of indices to get only a subset of images/labels for
            # the current shard file.
            file_indices = np.arange(start, start+files_per_shard, dtype=int)
            start = start + files_per_shard
            self.write_tfrecords_file(shard_path, self.train_filenames, self.train_labels, file_indices)

        # The remaining images will go in the final shard.
        file_indices = np.arange(start, self.num_files_train, dtype=int)
        if len(file_indices) > 0:
            final_shard_path = self.output_path / f'train-{self.num_shards_train}.tfrecords'
            self.write_tfrecords_file(final_shard_path, self.train_filenames, self.train_labels, file_indices)

        print(f'\nProcessed {self.counter} total images...')
        print(f'Number of training examples: {self.num_files_train}')
        print(f'TFRecords files saved to {str(self.output_path)}')

In [ ]:
converter = TFRecordsConverter()
converter.convert()

Processing training set TFRecords...
Processed 1000 images...
Processed 2000 images...
Processed 3000 images...
Processed 4000 images...
Processed 5000 images...
Processed 6000 images...
Processed 7000 images...
Processed 8000 images...

Processed 8960 total images...
Number of training examples: 8960
TFRecords files saved to tfrecords-output
